### Projeto Final de Spark - Avançado
#### Campanha Nacional de Vacinação contra Covid-19

Autor: Arthur Freitas Gomes

Turma: Semantix Academy - Big Data Engineer 04-21

Professor: Rodrigo Rebouças

##### Nível Avançado:

Replicar as visualizações do site “https://covid.saude.gov.br/”, porém acessando diretamente a API de Elastic.

Link oficial para todas as informações:  https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao

Informações para se conectar ao cluster:

    > URL https://imunizacao-es.saude.gov.br/desc-imunizacao
    > Nome do índice: desc-imunizacao
    > Credenciais de acesso
    > Usuário: imunizacao_public
    > Senha: qlto5t&7r_@+#Tlstigi

In [1]:
import requests
import pandas as pd
import time

In [2]:
#Acesso inicial para gravar '_scroll_id'
covid_API = requests.get('https://imunizacao-es.saude.gov.br/_search?scroll=1m',
                         auth=requests.auth.HTTPBasicAuth('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'),
                         params={"size":10000})

In [3]:
#covid_API.json()

In [4]:
#'_scroll_id' = id temporário que será utilizado para navegar nos scrolls com 10000 registros cada
covid_API.json()['_scroll_id']

'FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldGNoAxZOaHUxUUotT1FpS2N4M3dwaEFYM3J3AAAAAG2Pt2IWanNYbEN0UjlUUXVLVXdrcGo5cE43ZxZlR0trMDNRRlFreTlFRWtxel9vME1BAAAAAGRvjiwWeUtZa0E0b1ZTZlNjam5veUxfS1ZHdxYycGRZR2c4SVNXeWpFSkh6c0JjXzFBAAAAAG5qqMIWV3hBS3dIbGpTWkNDV0dBYWMwUUU0dw=='

In [5]:
#Número de registros por Scroll
len(covid_API.json()['hits']['hits'])

10000

In [6]:
#Scroll = 0 

count_scroll = 0

scroll_id = covid_API.json()['_scroll_id']
covid_API_scroll = requests.get('https://imunizacao-es.saude.gov.br/_search/scroll',
                                auth=requests.auth.HTTPBasicAuth('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'),
                                params={"scroll_id":scroll_id, "scroll":'1m'})

#recebe infomações HITS, seleciona dict _source e salva em um json
pd.DataFrame([x['_source'] for x in covid_API_scroll.json()['hits']['hits']]).to_json('api_covid/covid_scroll_'+str(count_scroll)+'.json')

In [7]:
%%time
#Loop para repetir processo anterior até hit  = 0

"""
Cada arquivo .json está com ~8Mb e temos 116.089.044 hits, ou seja, 11.608 arquivos, para o exemplo limitei a 10 arquivos
"""

while count_scroll < 10:
#while len(covid_API_scroll.json()['hits']['hits']) != 0:
    scroll_id = covid_API.json()['_scroll_id']
    covid_API_scroll = requests.get('https://imunizacao-es.saude.gov.br/_search/scroll', auth=requests.auth.HTTPBasicAuth('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'), params={"scroll_id":scroll_id, "scroll":'1m'})

    #covid_API_scroll.json()
    #covid_API_scroll.json()['hits']['hits']
    
    count_scroll += 1
    pd.DataFrame([x['_source'] for x in covid_API_scroll.json()['hits']['hits']]).to_json('api_covid/covid_scroll_'+str(count_scroll)+'.json')

CPU times: user 6.18 s, sys: 395 ms, total: 6.57 s
Wall time: 13.1 s


In [8]:
!ls api_covid/

covid_scroll_0.json   covid_scroll_3.json  covid_scroll_7.json
covid_scroll_1.json   covid_scroll_4.json  covid_scroll_8.json
covid_scroll_10.json  covid_scroll_5.json  covid_scroll_9.json
covid_scroll_2.json   covid_scroll_6.json


In [9]:
pd.read_json('api_covid/covid_scroll_0.json').head()

,estabelecimento_razaoSocial,vacina_dataAplicacao,vacina_grupoAtendimento_codigo,estabelecimento_valor,@timestamp,sistema_origem,vacina_lote,id_sistema_origem,estalecimento_noFantasia,paciente_endereco_coIbgeMunicipio,...,vacina_descricao_dose,vacina_fabricante_nome,vacina_categoria_codigo,paciente_endereco_uf,vacina_categoria_nome,redshift,vacina_nome,paciente_racaCor_valor,paciente_id,paciente_enumSexoBiologico
0,PREFEITURA MUNICIPAL DE PRAIA GRANDE,2021-03-20T00:00:00.000Z,203,2305658,2021-07-09T18:41:46.638Z,Novo PNI,210063,16341,UNIDADE DE SAUDE CENTRAL,421380,...,1ª Dose,FUNDACAO BUTANTAN,2,SC,Faixa Etária,v2,Covid-19-Coronavac-Sinovac/Butantan,BRANCA,cb93ec22465279865c1e2f12b45344226035fdf3c078f0...,M
1,MUNICIPIO DE BOA VISTA,2021-03-20T00:00:00.000Z,203,9193235,2021-07-09T18:41:46.638Z,Novo PNI,200278,16341,COORDENACAO MUNICIPAL DE IMUNIZACAO,140010,...,1ª Dose,FUNDACAO BUTANTAN,2,RR,Faixa Etária,v2,Covid-19-Coronavac-Sinovac/Butantan,PARDA,6e0f5e0f4fc5fb3bd7f8b85d2f61314a88b55311dd1b50...,M
2,PREFEITURA MUNICIPAL DE POUSO ALEGRE,2021-03-17T00:00:00.000Z,916,2211610,2021-07-09T18:41:46.638Z,Novo PNI,210018A,16341,UBS ISMAEL LIBANIO UBS PUERICULTURA P ALEGRE,313150,...,2ª Dose,FUNDACAO BUTANTAN,9,MG,Trabalhadores de Saúde,v2,Covid-19-Coronavac-Sinovac/Butantan,BRANCA,1e4b602c9c95f635dca23c9e27d7a29c853e39fe147afe...,M
3,PREFEITURA MUNICIPAL DE SAPUCAIA DO SUL,2021-03-20T00:00:00.000Z,203,3922448,2021-07-09T18:41:46.639Z,Novo PNI,210074,16341,VIGILANCIA EM SAUDE,432000,...,1ª Dose,FUNDACAO BUTANTAN,2,RS,Faixa Etária,v2,Covid-19-Coronavac-Sinovac/Butantan,BRANCA,01be20fe02b809aa41e7524e60146936ba2427c44dc50b...,F
4,PREF M MARANGUAPE,2021-02-18T00:00:00.000Z,301,2373750,2021-07-09T18:41:46.640Z,Novo PNI,202010034,16341,UBS MARIA FRANCINA DE ARAUJO,230770,...,2ª Dose,FUNDACAO BUTANTAN,3,CE,Pessoas de 60 anos ou mais institucionalizadas,v2,Covid-19-Coronavac-Sinovac/Butantan,PARDA,70645e8d7b3df120c746db62b3dabbf65b23bebbe00a06...,F


##### Próximos passos:

    > enviar para namenode;
    > criar visualizações.